<a href="https://colab.research.google.com/github/ibrahimhan07/DataMBG/blob/Code/Stemming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import os # Library untuk interaksi dengan sistem file, misal cek keberadaan file

# --- Konfigurasi Awal (SILAKAN SESUAIKAN BAGIAN INI SESUAI FILE ANDA) ---
# Nama file input CSV/Excel dari RapidMiner yang sudah di-cleansing.
# Pastikan file ini ada di folder yang sama dengan skrip Python ini,
# atau berikan path (alamat folder) lengkapnya.
input_file_name = "MBGLabel.csv"

# Nama file output CSV/Excel tempat hasil stemming akan disimpan.
output_file_name = "data_stemmed.csv"

# Nama kolom di file input Anda yang berisi teks yang sudah bersih
# (misalnya, setelah menghilangkan URL, simbol, dsb.).
# CONTOH: 'text_bersih', 'tweet_text_clean', 'teks_tweet'
text_column_name = "Sentimen;Text" # <--- GANTI SESUAI NAMA KOLOM TEKS ANDA!

# Nama kolom baru yang akan dibuat untuk menyimpan hasil stemming
new_stemmed_column_name = "teks_stemmed"

# --- Daftar Pengecualian (Exclusion List) ---
# Kata-kata dalam daftar ini TIDAK akan di-stem.
# Masukkan kata-kata kunci dari judul skripsi Anda yang tidak ingin berubah,
# serta variasi penting lainnya yang ingin Anda pertahankan.
# PASTIKAN SEMUA KATA DI SINI DALAM HURUF KECIL (lowercase).
exclusion_list = [
    "makan",
    "bergizi",
    "gratis",
    "program",

]

# --- Inisialisasi Stemmer ---
print("Memulai inisialisasi Sastrawi Stemmer (Algoritma Nazief & Adriani)...")
factory = StemmerFactory()
stemmer = factory.create_stemmer()
print("Sastrawi Stemmer berhasil diinisialisasi.")

# --- Membaca Data dari File Input ---
print(f"Mencoba membaca data dari file: {input_file_name}...")
try:
    if not os.path.exists(input_file_name):
        raise FileNotFoundError(f"File '{input_file_name}' tidak ditemukan. Pastikan nama file dan path-nya benar.")

    # Otomatis deteksi format file (CSV atau Excel)
    if input_file_name.lower().endswith('.csv'):
        df = pd.read_csv(input_file_name)
    elif input_file_name.lower().endswith(('.xlsx', '.xls')):
        df = pd.read_excel(input_file_name)
    else:
        raise ValueError("Format file input tidak didukung. Harap gunakan .csv, .xlsx, atau .xls.")

    print(f"Data berhasil dibaca. Jumlah baris: {len(df)}.")
    print(f"Daftar kolom yang ditemukan: {df.columns.tolist()}")

    # Memastikan kolom teks yang dituju ada
    if text_column_name not in df.columns:
        raise KeyError(f"Kolom '{text_column_name}' tidak ditemukan dalam file input. Harap periksa kembali nama kolomnya di file Anda.")

except FileNotFoundError as fnfe:
    print(f"ERROR: {fnfe}")
    print("Pastikan file input ada di folder yang sama atau berikan path lengkap.")
    exit() # Keluar dari skrip jika file tidak ditemukan
except KeyError as ke:
    print(f"ERROR: {ke}")
    print("Pastikan 'text_column_name' sesuai dengan nama kolom di file Anda.")
    exit()
except Exception as e:
    print(f"ERROR: Terjadi kesalahan saat membaca file: {e}")
    exit()

# --- Fungsi untuk Menerapkan Stemming dengan Pengecualian ---
def apply_sastrawi_stemming(text):
    # Cek apakah nilai adalah string dan bukan NaN (Not a Number)
    if pd.isna(text) or not isinstance(text, str):
        return text # Kembalikan nilai asli jika kosong/bukan string

    # Ubah teks ke huruf kecil untuk konsistensi pengecekan (penting!)
    text_lower = text.lower()

    # Tokenisasi sederhana (Anda bisa menggunakan NLTK/spaCy jika perlu tokenisasi yang lebih canggih)
    words = text_lower.split() # Memisahkan kalimat menjadi kata-kata berdasarkan spasi

    stemmed_words = []
    for word in words:
        # Jika kata (setelah di-lowercase) ada dalam daftar pengecualian, biarkan apa adanya
        if word in exclusion_list:
            stemmed_words.append(word)
        else:
            # Lakukan stemming untuk kata-kata lainnya
            stemmed_words.append(stemmer.stem(word))

    # Gabungkan kembali kata-kata yang sudah di-stem menjadi sebuah kalimat
    return " ".join(stemmed_words)

# --- Melakukan Proses Stemming ---
print(f"Memulai proses stemming pada kolom '{text_column_name}'...")
# Membuat kolom baru untuk menyimpan hasil stemming
df[new_stemmed_column_name] = df[text_column_name].apply(apply_sastrawi_stemming)
print("Proses stemming selesai.")
print(f"Kolom baru '{new_stemmed_column_name}' telah ditambahkan.")

# --- Menyimpan Hasil ke File Output ---
print(f"Menyimpan hasil stemming ke file: {output_file_name}...")
try:
    if output_file_name.lower().endswith('.csv'):
        df.to_csv(output_file_name, index=False) # index=False agar tidak menyimpan nomor indeks sebagai kolom baru
    elif output_file_name.lower().endswith(('.xlsx', '.xls')):
        df.to_excel(output_file_name, index=False)
    else:
        print("Peringatan: Format file output tidak dikenal, akan disimpan sebagai CSV.")
        df.to_csv(output_file_name + ".csv", index=False)

    print(f"Data yang sudah di-stem berhasil disimpan di '{output_file_name}'.")
    print("Anda sekarang dapat mengimpor file ini ke RapidMiner untuk langkah selanjutnya.")

except Exception as e:
    print(f"ERROR: Terjadi kesalahan saat menyimpan file: {e}")

Memulai inisialisasi Sastrawi Stemmer (Algoritma Nazief & Adriani)...
Sastrawi Stemmer berhasil diinisialisasi.
Mencoba membaca data dari file: MBGLabel.csv...
Data berhasil dibaca. Jumlah baris: 10001.
Daftar kolom yang ditemukan: ['Sentimen;Text']
Memulai proses stemming pada kolom 'Sentimen;Text'...
Proses stemming selesai.
Kolom baru 'teks_stemmed' telah ditambahkan.
Menyimpan hasil stemming ke file: data_stemmed.csv...
Data yang sudah di-stem berhasil disimpan di 'data_stemmed.csv'.
Anda sekarang dapat mengimpor file ini ke RapidMiner untuk langkah selanjutnya.
